In [1]:
import numpy as np
import matplotlib
import datetime
matplotlib.use('Agg')
import matplotlib.pyplot as plt

Replication Code

In [26]:
def createnosignaldata(n,d):
    """
    Data points are random Gaussian vectors.
    Class labels are random and uniform
    """
    X_train = np.random.normal(0,1,(n,d+1))
    X_train[:,d] = np.sign(X_train[:,d])
    X_holdout = np.random.normal(0,1,(n,d+1))
    X_holdout[:,d] = np.sign(X_holdout[:,d])
    X_test = np.random.normal(0,1,(n,d+1))
    X_test[:,d] = np.sign(X_test[:,d])
    return X_train, X_holdout, X_test


def createhighsignaldata(n,d):
    """
    Data points are random Gaussian vectors.
    Class labels are random and uniform
    First nbiased are biased with bias towards the class label
    """
    X_train = np.random.normal(0,1,(n,d+1))
    X_train[:,d] = np.sign(X_train[:,d])
    X_holdout = np.random.normal(0,1,(n,d+1))
    X_holdout[:,d] = np.sign(X_holdout[:,d])
    X_test = np.random.normal(0,1,(n,d+1))
    X_test[:,d] = np.sign(X_test[:,d])

    # Add correlation with the sign
    nbiased = 20
    bias = 6.0/np.sqrt(n)
    b = np.zeros(nbiased)
    for i in range(n):
        b[0:nbiased] = bias*X_holdout[i,d]
        X_holdout[i,range(0,nbiased)] = np.add(X_holdout[i,range(0,nbiased)], b)

    for i in range(n):
        b[0:nbiased] = bias*X_train[i,d]
        X_train[i,range(0,nbiased)] = np.add(X_train[i,range(0,nbiased)], b)

    for i in range(n):
        b[0:nbiased] = bias*X_test[i,d]
        X_test[i,range(0,nbiased)] = np.add(X_test[i,range(0,nbiased)], b)

    return X_train, X_holdout, X_test

def runClassifier(n,d,krange, X_train,X_holdout,X_test, tolerance=None, threshold=None):
    """
    Variable selection and basic boosting on synthetic data. Variables
    with largest correlation with target are selected first.
    """
    # Compute values on the standard holdout
    if tolerance == None:
        tolerance = 1.0/np.sqrt(n)
    if threshold == None:
        threshold = 4.0/np.sqrt(n)

    vals = []
    trainanswers = np.dot(X_train[:,range(0,d)].T,X_train[:,d])/n
    holdoutanswers = np.dot(X_holdout[:,range(0,d)].T,X_holdout[:,d])/n
    trainpos = trainanswers > 1.0/np.sqrt(n)
    holdopos = holdoutanswers > 1.0/np.sqrt(n)
    trainneg = trainanswers < -1.0/np.sqrt(n)
    holdoneg = holdoutanswers < -1.0/np.sqrt(n)
    selected = (trainpos & holdopos) | (trainneg & holdoneg)
    trainanswers[~selected] = 0
    sortanswers = np.abs(trainanswers).argsort()
    for k in krange:
        weights = np.zeros(d+1)
        topk = sortanswers[-k:]
        weights[topk] = np.sign(trainanswers[topk])
        ftrain = 1.0*np.count_nonzero(np.sign(np.dot(X_train,weights)) == X_train[:,d])/n
        fholdout = 1.0*np.count_nonzero(np.sign(np.dot(X_holdout,weights)) == X_holdout[:,d])/n
        ftest = 1.0*np.count_nonzero(np.sign(np.dot(X_test,weights)) == X_test[:,d])/n
        if k == 0:
            vals.append([0.5,0.5,0.5])
        else:
            vals.append([ftrain,fholdout,ftest])

    # Compute values using Thresholdout
    noisy_vals = []
    trainanswers = np.dot(X_train[:,range(0,d)].T,X_train[:,d])/n
    holdoutanswers = np.dot(X_holdout[:,range(0,d)].T,X_holdout[:,d])/n
    diffs = np.abs(trainanswers - holdoutanswers)
    noise = np.random.normal(0,tolerance,d)
    abovethr = diffs > threshold + noise
    holdoutanswers[~abovethr] = trainanswers[~abovethr]
    holdoutanswers[abovethr] = (holdoutanswers+np.random.normal(0,tolerance,d))[abovethr]
    trainpos = trainanswers > 1.0/np.sqrt(n)
    holdopos = holdoutanswers > 1.0/np.sqrt(n)
    trainneg = trainanswers < -1.0/np.sqrt(n)
    holdoneg = holdoutanswers < -1.0/np.sqrt(n)
    selected = (trainpos & holdopos) | (trainneg & holdoneg)
    trainanswers[~selected] = 0
    sortanswers = np.abs(trainanswers).argsort()
    for k in krange:
        weights = np.zeros(d+1)
        topk = sortanswers[-k:]
        weights[topk] = np.sign(trainanswers[topk])
        ftrain = 1.0*np.count_nonzero(np.sign(np.dot(X_train,weights)) == X_train[:,d])/n
        fholdout = 1.0*np.count_nonzero(np.sign(np.dot(X_holdout,weights)) == X_holdout[:,d])/n
        if abs(ftrain-fholdout) < threshold + np.random.normal(0,tolerance):
            fholdout = ftrain
        else:
            fholdout += np.random.normal(0,tolerance)
        ftest = 1.0*np.count_nonzero(np.sign(np.dot(X_test,weights)) == X_test[:,d])/n
        if k == 0:
            noisy_vals.append([0.5,0.5,0.5])
        else:
            noisy_vals.append([ftrain,fholdout,ftest])

    return vals, noisy_vals

def plot1(x,mean,std,plotname,plottitle,legend_pos=2):
    fig = plt.figure(1,figsize=(6.5,4))
    plt.title(plottitle,fontsize='14')
    plt.ylabel('accuracy',fontsize='14')
    plt.xlabel('number of variables',fontsize='14')
    plt.axis([x[0], x[-1], 0.45, 0.75])
    plt.plot(x,mean[:,0],'b^-',label='training')
    plt.fill_between(x,mean[:,0]-std[:,0],mean[:,0]+std[:,0],alpha=0.5,edgecolor='#B2B2F5',facecolor='#B2B2F5',linestyle='dashdot')
    plt.plot(x,mean[:,1],'x-',label='holdout',color='#006600')
    plt.fill_between(x,mean[:,1]-std[:,1],mean[:,1]+std[:,1],alpha=0.5,edgecolor='#33CC33',facecolor='#CCFFCC',linestyle='dashdot')
    plt.plot(x,mean[:,2],'r|-',label='fresh')
    plt.fill_between(x,mean[:,2]-std[:,2],mean[:,2]+std[:,2],alpha=0.5,edgecolor='#CC4F1B',facecolor='#FF9848',linestyle='dashdot')
    plt.legend(loc=legend_pos,prop={'size':12})
    plt.tight_layout()
    plt.savefig(plotname+".pdf")
    plt.close()

def avgout(vallist):
    """ entry-wise average of a list of matrices """
    r = len(vallist)
    A = 0
    if r > 0:
        for B in vallist:
            A += (1.0/r) * B
    return A


def stddev(vallist):
    """ entry-wise standard deviation of a list of matrices """
    r = len(vallist)
    mean = avgout(vallist)
    A = 0
    if r > 0:
        for B in vallist:
            A += (1.0/r) * (B - mean)**2
    return np.sqrt(A)


def repeatexp(n,d,krange,reps,datafn, tolerance=None, threshold=None):
    """ Repeat experiment specified by fn for reps steps """
    vallist = []
    vallist2 = []
    for r in range(0,reps):
        print ("Repetition:", r)
        X_train,X_holdout,X_test = datafn(n,d)
        vals,vals2 = runClassifier(n,d,krange,X_train,X_holdout,X_test, tolerance, threshold)
        vallist.append(np.array(vals))
        vallist2.append(np.array(vals2))
    return vallist, vallist2


def runandplotsummary(n,d,krange,reps,datafn,plotname, tolerance=None, threshold=None):
    vallist,vallist2 = repeatexp(n,d,krange,reps,datafn, tolerance, threshold)
    mean = avgout(vallist)
    std = stddev(vallist)
    mean2 = avgout(vallist2)
    std2 = stddev(vallist2)
    f = open(plotname+".txt",'w')
    f.write(str(mean))
    f.write("\n")
    f.write(str(std))
    f.write("\n")
    f.write(str(mean2))
    f.write("\n")
    f.write(str(std2))
    f.close()
    plot1(krange,mean,std,plotname+"-std","Standard holdout")
    plot1(krange,mean2,std2,plotname+"-thr","Thresholdout")

In [30]:
reps = 100
n, d = 10000, 10000
krange = [0,10,20,30,45,70,100,150,200,250,300,400,500] # number of variables

today = datetime.datetime.now()
timestamp = str(today.month)+str(today.day)+"."+str(today.hour)+str(today.minute)

In [ ]:

# Experiment 1:
# No correlations
plotname = "plot-"+timestamp+"-"+str(n)+"-"+str(d)+"-"+str(reps)+"-nosignal"
runandplotsummary(n,d,krange,reps,createnosignaldata,plotname)



# Experiment 2:
# Some variables are correlated
plotname = "plot-"+timestamp+"-"+str(n)+"-"+str(d)+"-"+str(reps)+"-highsignal"
runandplotsummary(n,d,krange,reps,createhighsignaldata,plotname)

Sim 1

Dependence within holdout

In [31]:
def createrepeatedholdout(n, d):
    """
    Repeat each value 10 times
    """
    X_train = np.random.normal(0,1,(int(n / 10),d+1))
    X_train_full = np.repeat(X_train, 10, axis=0)
    X_train_full[:,d] = np.sign(X_train_full[:,d])
    X_holdout = np.random.normal(0,1,(int(n / 10),d+1))
    X_holdout_full = np.repeat(X_holdout, 10, axis=0)
    X_holdout_full[:,d] = np.sign(X_holdout_full[:,d])
    X_test = np.random.normal(0,1,(n,d+1))
    X_test[:,d] = np.sign(X_test[:,d])
    return X_train_full, X_holdout_full, X_test

def createrepeatedholdout2(n, d):
    """
    Repeat each value 50 times
    """
    X_train = np.random.normal(0,1,(int(n / 50),d+1))
    X_train_full = np.repeat(X_train, 50, axis=0)
    X_train_full[:,d] = np.sign(X_train_full[:,d])
    X_holdout = np.random.normal(0,1,(int(n / 50),d+1))
    X_holdout_full = np.repeat(X_holdout, 50, axis=0)
    X_holdout_full[:,d] = np.sign(X_holdout_full[:,d])
    X_test = np.random.normal(0,1,(n,d+1))
    X_test[:,d] = np.sign(X_test[:,d])
    return X_train_full, X_holdout_full, X_test

def createrepeatedholdout3(n, d):
    """
    Repeat each value 2 times
    """
    X_train = np.random.normal(0,1,(int(n / 2),d+1))
    X_train_full = np.repeat(X_train, 2, axis=0)
    X_train_full[:,d] = np.sign(X_train_full[:,d])
    X_holdout = np.random.normal(0,1,(int(n / 2),d+1))
    X_holdout_full = np.repeat(X_holdout, 2, axis=0)
    X_holdout_full[:,d] = np.sign(X_holdout_full[:,d])
    X_test = np.random.normal(0,1,(n,d+1))
    X_test[:,d] = np.sign(X_test[:,d])
    return X_train_full, X_holdout_full, X_test

plotname = "sim1plot-"+timestamp+"-"+str(n)+"-"+str(d)+"-"+str(reps)+"-repeatedholdout10"
runandplotsummary(n,d,krange,reps,createrepeatedholdout,plotname)

# plotname = "plot-"+timestamp+"-"+str(n)+"-"+str(d)+"-"+str(reps)+"-repeatedholdout50"
# runandplotsummary(n,d,krange,reps,createrepeatedholdout2,plotname)

plotname = "sim1plot-"+timestamp+"-"+str(n)+"-"+str(d)+"-"+str(reps)+"-repeatedholdout2"
runandplotsummary(n,d,krange,reps,createrepeatedholdout3,plotname)

Repetition: 0
Repetition: 1
Repetition: 2
Repetition: 3
Repetition: 4
Repetition: 5
Repetition: 6
Repetition: 7
Repetition: 8
Repetition: 9
Repetition: 10
Repetition: 11
Repetition: 12
Repetition: 13
Repetition: 14
Repetition: 15
Repetition: 16
Repetition: 17
Repetition: 18
Repetition: 19
Repetition: 20
Repetition: 21
Repetition: 22
Repetition: 23
Repetition: 24
Repetition: 25
Repetition: 26
Repetition: 27
Repetition: 28
Repetition: 29
Repetition: 30
Repetition: 31
Repetition: 32
Repetition: 33
Repetition: 34
Repetition: 35
Repetition: 36
Repetition: 37
Repetition: 38
Repetition: 39
Repetition: 40
Repetition: 41
Repetition: 42
Repetition: 43
Repetition: 44
Repetition: 45
Repetition: 46
Repetition: 47
Repetition: 48
Repetition: 49
Repetition: 50
Repetition: 51
Repetition: 52
Repetition: 53
Repetition: 54
Repetition: 55
Repetition: 56
Repetition: 57
Repetition: 58
Repetition: 59
Repetition: 60
Repetition: 61
Repetition: 62
Repetition: 63
Repetition: 64
Repetition: 65
Repetition: 66
Repet

Sim 2

Change runClassifier to accept different tau and threshold

In [32]:
tolerances = [1.0/np.sqrt(n), 10.0/np.sqrt(n), 0.1/np.sqrt(n),]
thresholds = [4.0/np.sqrt(n), 40.0/np.sqrt(n), 0.4/np.sqrt(n)]

for tol in tolerances:
    for thresh in thresholds:
        plotname = "sim2plot-"+str(tol)+"tol"+str(thresh)+"thresh"+timestamp+"-"+str(n)+"-"+str(d)+"-"+str(reps)+"-params"
        runandplotsummary(n,d,krange,reps,createnosignaldata,plotname, tolerance=tol, threshold=thresh)


Repetition: 0
Repetition: 1
Repetition: 2
Repetition: 3
Repetition: 4
Repetition: 5
Repetition: 6
Repetition: 7
Repetition: 8
Repetition: 9
Repetition: 10
Repetition: 11
Repetition: 12
Repetition: 13
Repetition: 14
Repetition: 15
Repetition: 16
Repetition: 17
Repetition: 18
Repetition: 19
Repetition: 20
Repetition: 21
Repetition: 22
Repetition: 23
Repetition: 24
Repetition: 25
Repetition: 26
Repetition: 27
Repetition: 28
Repetition: 29
Repetition: 30
Repetition: 31
Repetition: 32
Repetition: 33
Repetition: 34
Repetition: 35
Repetition: 36
Repetition: 37
Repetition: 38
Repetition: 39
Repetition: 40
Repetition: 41
Repetition: 42
Repetition: 43
Repetition: 44
Repetition: 45
Repetition: 46
Repetition: 47
Repetition: 48
Repetition: 49
Repetition: 50
Repetition: 51
Repetition: 52
Repetition: 53
Repetition: 54
Repetition: 55
Repetition: 56
Repetition: 57
Repetition: 58
Repetition: 59
Repetition: 60
Repetition: 61
Repetition: 62
Repetition: 63
Repetition: 64
Repetition: 65
Repetition: 66
Repet

Sim 3

Continuous outcome